In [1]:
import os
os.chdir("..")
import scanpy as sc
import numpy as np
import datasets as DS

d:\anaconda3\envs\scanpy\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
adata_ref = sc.read_visium(path = "data/Human_DLPFC_151673/")
adata_ref.var_names_make_unique()
adata_ref.var["mt"] = adata_ref.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(adata_ref, qc_vars = ["mt"], inplace = True)
sc.pp.filter_genes(adata_ref, min_cells = 3)

d:\anaconda3\envs\scanpy\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [3]:
adata_q = sc.read_visium(path = "data/Human_DLPFC_151676/")
adata_q.var_names_make_unique()
adata_q.var["mt"] = adata_q.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(adata_q, qc_vars = ["mt"], inplace = True)
sc.pp.filter_genes(adata_q, min_cells = 3)

d:\anaconda3\envs\scanpy\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [4]:
def prefilter_specialgenes(adata):
    id_tmp1 = np.asarray([not str(name).startswith("ERCC") for name in adata.var_names], dtype = bool)
    id_tmp2 = np.asarray([not str(name).startswith("MT-") for name in adata.var_names], dtype = bool)
    id_tmp = np.logical_and(id_tmp1, id_tmp2)
    adata._inplace_subset_var(id_tmp)
prefilter_specialgenes(adata_ref)
prefilter_specialgenes(adata_q)

In [5]:
DS.adata2seurat(adata_ref, "human_DLPFC")
! Rscript spagene.R human_DLPFC 300
spagene1 = DS.find_spa("human_DLPFC" + "/spagene.txt")

Loading required package: ggplot2
Loading required package: patchwork


In [6]:
DS.adata2seurat(adata_q, "human_DLPFC")
! Rscript spagene.R human_DLPFC 300
spagene2 = DS.find_spa("human_DLPFC" + "/spagene.txt")

Loading required package: ggplot2
Loading required package: patchwork


In [7]:
spagene = list(set(spagene1) & set(spagene2))

In [11]:
import pickle
with open("human_DLPFC/spagene.pkl", "wb") as f:
    pickle.dump(spagene, f)

In [ ]:
ldp_train = DS.LabeledDataUnit(adata_ref, 2.0)
ldp_train.get_count(gene = spagene)
ldp_train.calc_distance()
ldp_train.get_data()

In [ ]:
from imb_torch2 import knn, to_device
_, _, BD_neg_index = knn(ldp_train, k = 20)
ldp_train.pop(BD_neg_index)
ldp_train.mirror_copy()

In [ ]:
ldp_train = to_device(ldp_train, device = 0)

In [ ]:
from model import nn_model2
model = nn_model2.NNClassifier(len(spagene)*2, hidden_sizes=[512, 64, 16], max_iter = 30, batch_size = 32, gpu_id = 0)
model.fit(ldp_train)

In [ ]:
model.predict()